In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_75601: Some(http://gw02.itversity.com:4044)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_75601

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home + "/dataSets/spark-guide/data/retail-data/by-day/*.csv"

path = /user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv


/user/kranthidr/dataSets/spark-guide/data/retail-data/by-day/*.csv

In [6]:
//spark.conf.set("spark.sql.shuffle.partitions", "5")

Name: Syntax Error.
Message: 
StackTrace: 

In [7]:
val staticDataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(path)

staticDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [8]:
staticDataFrame.rdd.getNumPartitions

10

In [9]:
staticDataFrame.repartition(20)
staticDataFrame.count()

541909

In [10]:
staticDataFrame.rdd.getNumPartitions

10

In [11]:
staticDataFrame.coalesce(5)
staticDataFrame.count()

541909

In [12]:
staticDataFrame.rdd.getNumPartitions

10

In [13]:
staticDataFrame.createOrReplaceTempView("retail_data")
val staticSchema = staticDataFrame.schema

staticSchema = StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))


StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))

In [14]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
print(staticDataFrame.explain())

== Physical Plan ==
*(1) FileScan csv [InvoiceNo#10,StockCode#11,Description#12,Quantity#13,InvoiceDate#14,UnitPrice#15,CustomerID#16,Country#17] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/data/retail-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,InvoiceDate:timestamp,Un...
()

In [16]:
staticDataFrame.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [17]:
import org.apache.spark.sql.functions.{window, column, desc, col}

staticDataFrame
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")
  .groupBy($"CustomerId", window($"InvoiceDate", "1 day"))
  .sum("total_cost")
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   12622.0|[2011-04-20 20:00...|180.09999999999997|
|   14527.0|[2011-05-16 20:00...|             -9.84|
|   16013.0|[2011-09-29 20:00...|             669.6|
|   15249.0|[2011-08-23 20:00...| 696.5600000000001|
|   13402.0|[2011-08-03 20:00...|            312.95|
+----------+--------------------+------------------+
only showing top 5 rows



In [18]:
import org.apache.spark.sql.functions.{date_format, col}

val preppedDataFrame = staticDataFrame
  .na.fill(0)
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))
  .coalesce(5).cache()

preppedDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [19]:
preppedDataFrame.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WATER TRA..

In [20]:
val trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
val testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

trainDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]
testDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [21]:
println(trainDataFrame.count(), testDataFrame.count())

(245903,296006)


In [22]:
import org.apache.spark.ml.feature.StringIndexer
val indexer = new StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")

indexer = strIdx_39a4a02a6ea0


strIdx_39a4a02a6ea0

In [23]:
import org.apache.spark.ml.feature.OneHotEncoder
val encoder = new OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")

encoder = oneHot_566bfc001fe4


oneHot_566bfc001fe4

In [24]:
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler().setInputCols(Array("UnitPrice", "Quantity", "day_of_week_encoded"))
  .setOutputCol("features")

vectorAssembler = vecAssembler_393df095db00


vecAssembler_393df095db00

In [25]:
import org.apache.spark.ml.Pipeline
val transformationPipeline = new Pipeline().setStages(Array(indexer, encoder, vectorAssembler))

transformationPipeline = pipeline_086db9c41f64


pipeline_086db9c41f64

In [26]:
val fittedPipeline = transformationPipeline.fit(trainDataFrame)

fittedPipeline = pipeline_086db9c41f64


pipeline_086db9c41f64

In [27]:
val transformedTraining = fittedPipeline.transform(trainDataFrame)

transformedTraining = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [28]:
transformedTraining.cache()

[InvoiceNo: string, StockCode: string ... 10 more fields]

In [29]:
import org.apache.spark.ml.clustering.KMeans
val kmeans = new KMeans().setK(20).setSeed(1)

kmeans = kmeans_a07790bda9ec


kmeans_a07790bda9ec

In [30]:
val kmModel = kmeans.fit(transformedTraining)

kmModel = kmeans_a07790bda9ec


kmeans_a07790bda9ec

In [31]:
val transformedTest = fittedPipeline.transform(testDataFrame)

transformedTest = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [32]:
transformedTest.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = false)
 |-- day_of_week_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
kmModel.computeCost(transformedTest)

5.276368264031295E8

In [34]:
kmModel.computeCost(transformedTraining)

7.749737172489917E7